In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import re
import string
import nltk
from nltk.corpus import stopwords
from argparse import Namespace
arg = Namespace()



In [2]:
df = pd.read_csv("../data/Rest-Mex_2025_train.csv")
df.head()

,Title,Review,Polarity,Town,Region,Type
0,Mi Lugar Favorito!!!!,Excelente lugar para comer y pasar una buena n...,5.0,Sayulita,Nayarit,Restaurant
1,lugares interesantes para visitar,"andar mucho, así que un poco difícil para pers...",4.0,Tulum,QuintanaRoo,Attractive
2,No es el mismo Dreams,"Es nuestra cuarta visita a Dreams Tulum, elegi...",3.0,Tulum,QuintanaRoo,Hotel
3,un buen panorama cerca de CancÃºn,"Estando en CancÃºn, fuimos al puerto y tomamos...",4.0,Isla_Mujeres,QuintanaRoo,Attractive
4,El mejor,Es un lugar antiguo y por eso me encanto tiene...,5.0,Patzcuaro,Michoacan,Hotel


### Preprocesado

In [3]:
def arregla_mojibake(texto):
    try:
        return texto.encode('latin1').decode('utf-8')
    except:
        return texto  # Si falla la conversión, deja el texto igual

df['Title'] = df['Title'].fillna('').apply(arregla_mojibake)
df['Review'] = df['Review'].fillna('').apply(arregla_mojibake)

In [5]:
stopwords_es = set(stopwords.words('spanish'))

def limpiar_texto(texto):
    texto = texto.lower()  # Minúsculas
    texto = re.sub(r'[^a-záéíóúñü\s]', '', texto)  # Quitar puntuación y caracteres raros
    palabras = texto.split()
    palabras = [palabra for palabra in palabras if palabra not in stopwords_es]  # Remover stopwords
    return ' '.join(palabras)

df['Texto_Limpio'] = (df['Title'].fillna('') + ' ' + df['Review'].fillna('')).apply(limpiar_texto)

In [7]:
df_targetType = pd.concat([df["Texto_Limpio"], df["Type"]], axis = 1)
df_targetType.head()

,Texto_Limpio,Type
0,lugar favorito excelente lugar comer pasar bue...,Restaurant
1,lugares interesantes visitar andar así difícil...,Attractive
2,mismo dreams cuarta visita dreams tulum elegim...,Hotel
3,buen panorama cerca cancún cancún puerto tomam...,Attractive
4,mejor lugar antiguo encanto área juegos gigant...,Hotel


In [24]:
# Obtenemos un Label Encoder y un Vectorizer TFIDF
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

# Label Encoder
le = LabelEncoder()
le.fit(df_targetType["Type"])

# Vectorizer TFIDF
arg.tfidf_max_features = 2500
arg.tfidf_ngram_range = (1, 2)
arg.token_pattern=r'(?u)\b[^\d\W]+\b'
tfidf_vectorizer = TfidfVectorizer(max_features=arg.tfidf_max_features, 
                                   ngram_range=arg.tfidf_ngram_range, 
                                   token_pattern=arg.token_pattern)
tfidf_vectorizer.fit(df_targetType["Texto_Limpio"])

TfidfVectorizer(max_features=2500, ngram_range=(1, 2),
                token_pattern='(?u)\\b[^\\d\\W]+\\b')

In [25]:
class restaurantTargetTipo(Dataset):
    def __init__(self, data : pd.DataFrame, vectorizer, label_encoder,
                 use_pca = False, pca = None, n_components = 3):
        self.data = data
        self.n_samples = len(data)
        
        # Transform text to TF-IDF vectors
        tfidf_matrix = vectorizer.transform(data["Texto_Limpio"])

        if use_pca:
            self.X = torch.tensor(pca.transform(tfidf_matrix.toarray()), dtype=torch.float32)
            self.fitted_pca = pca
        else:
            self.X = torch.tensor(tfidf_matrix.toarray(), dtype=torch.float32)
            self.fitted_pca = None
        
        # Transform labels to numbers
        self.y = torch.tensor(label_encoder.transform(data["Type"]),
                              dtype=torch.long)

    def __len__(self):
        return self.n_samples

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
arg.use_pca = True
arg.pca_n_components = 3
arg.pca = PCA(n_components=arg.pca_n_components)

dataset = restaurantTargetTipo(df_targetType, tfidf_vectorizer, le,
                               use_pca=arg.use_pca, pca=arg.pca,
                               n_components=arg.pca_n_components)

AttributeError: 'PCA' object has no attribute 'components_'